In [1]:
# Model sieci neuronowej

import numpy as np


def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1.0 - sigmoid(z))

def train(X, y, n_hidden1, n_hidden2, learning_rate, n_iter):
    m, n_input = X.shape
    # Struktura (W - węzły, b - bias)
    W1 = np.random.randn(n_input, n_hidden1)
    b1 = np.zeros((1, n_hidden1))
    W2 = np.random.randn(n_hidden1, n_hidden2)
    b2 = np.zeros((1, n_hidden2))
    W3 = np.random.randn(n_hidden2, 2)
    b3 = np.zeros((1,2))
    for i in range(1, n_iter + 1):
        # Przejscie po sieci
        Z2 = np.matmul(X, W1) + b1
        A2 = sigmoid(Z2)
        Z3 = np.matmul(A2, W2) + b2
        A3 = sigmoid(Z3)
        Z4 = np.matmul(A3, W3) + b3
        A4 = Z4
        # Backpropagacja
        ## Koniec
        dZ4 = A4 - y
        dW3 = np.matmul(A3.T, dZ4)
        db3 = np.sum(dZ4, axis = 0, keepdims = True)
        ## Hidden 2
        dZ3 = np.matmul(dZ4, W3.T) * sigmoid_derivative(Z3)
        dW2 = np.matmul(A2.T, dZ3)
        db2 = np.sum(dZ3, axis = 0, keepdims = True)
        ## Hidden 1
        dZ2 = np.matmul(dZ3, W2.T) * sigmoid_derivative(Z2)
        dW1 = np.matmul(X.T, dZ2)
        db1 = np.sum(dZ2, axis = 0)
        # Poprawa sieci
        W3 = W3 - learning_rate * dW3 / m
        b3 = b3 - learning_rate * db3 / m
        W2 = W2 - learning_rate * dW2 / m
        b2 = b2 - learning_rate * db2 / m
        W1 = W1 - learning_rate * dW1 / m
        b1 = b1 - learning_rate * db1 / m
        # kontrola
        if i % 100 == 0:
            cost = np.mean((y - A4) ** 2)
            print('Iteracja: %i, strata: %f' %(i, cost))
    model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3}
    return model

def predict(x, model):
    W1 = model['W1']
    b1 = model['b1']
    W2 = model['W2']
    b2 = model['b2']
    W3 = model['W3']
    b3 = model['b3']
    A2 = sigmoid(np.matmul(x, W1) + b1)
    A3 = sigmoid(np.matmul(A2, W2) + b2)
    A4 = np.matmul(A3,W3) + b3
    return A4


        

In [2]:
# Klasy graczy i ich metody

import random

class dealer:
    deck = []
    hand = 0
    ace = False
    
    def shuffleDeck(self):
        self.deck = []
        self.hand = 0
        self.ace = False
        for i in range(2,12):
            for j in range(4):
                if i == 10:
                    for k in range(4):
                        self.deck.append(i)
                else:
                    self.deck.append(i)
        random.shuffle(self.deck)
    
    def giveCard(self):
        card = self.deck[0]
        self.deck.pop(0)
        return card
    
    def drawOne(self):
        card = self.giveCard()
        if self.ace and card == 11:
            self.hand += 1
        elif self.ace and self.hand + card > 21:
            self.hand += card - 10
            self.ace = False
        elif not self.ace and card == 11 and self.hand <= 10:
            self.ace = True
            self.hand += card
        else:
            self.hand += card
        return card
    
    def drawTillFin(self):
        while(self.hand < 17):
            card = self.drawOne()
        return self.hand


class player:
    hand = 0
    ace = 0
    
    def __init__(self, dealer, model = None, algorytm = None):
        self.dealer = dealer
        self.model = model
        self.algorytm = algorytm
    
    def isBusted(self):
        if(self.hand > 21):
            return True
        return False
    
    def move(self, trueValue):
        decision = None
        if(self.isBusted()):
            decision = -1
        elif(self.model != None):
            prediction = predict(np.array([self.hand, self.dealer.hand, self.ace, trueValue]), self.model)
            if prediction[0][0] > prediction[0][1]:
                decision = 1
            else:
                decision = 0
        else:
            decision = self.algorytm[str(self.hand)][str(self.dealer.hand)][str(int(self.ace))][str(trueValue)]
        
        return decision
    
    def hit(self):
        card = self.dealer.giveCard()
        if self.ace and card == 11:
            self.hand += 1
        elif self.ace and self.hand + card > 21:
            self.hand += card - 10
            self.ace = False
        elif not self.ace and card == 11 and self.hand <= 10:
            self.ace = True
            self.hand += card
        else:
            self.hand += card
        return card
    
    def reset(self):
        self.hand = 0
        self.ace = 0
    


            
    
        

In [3]:
# Trening modelu przy 1000 iteracji uczenia

import csv

train_in = np.loadtxt('learning_in.csv', skiprows=1, delimiter=',')
train_out = np.loadtxt('learning_out.csv', skiprows=1, delimiter=',')

print('Trening modelu 1:')
print()

hidden_nodes = 20
hidden_nodes_2 = 10
learn_rate = 0.1
num_of_iter = 1000

model_1 = train(train_in, train_out, hidden_nodes, hidden_nodes_2, learn_rate, num_of_iter)



Trening modelu 1:

Iteracja: 100, strata: 0.184134
Iteracja: 200, strata: 0.162713
Iteracja: 300, strata: 0.153808
Iteracja: 400, strata: 0.149614
Iteracja: 500, strata: 0.146666
Iteracja: 600, strata: 0.144096
Iteracja: 700, strata: 0.141722
Iteracja: 800, strata: 0.139516
Iteracja: 900, strata: 0.137473
Iteracja: 1000, strata: 0.135573


In [4]:
# Trening modelu przy 800 iteracji uczenia

print('Trening modelu 2:')
print()

hidden_nodes = 10
hidden_nodes_2 = 6
learn_rate = 0.15
num_of_iter = 800

model_2 = train(train_in, train_out, hidden_nodes, hidden_nodes_2, learn_rate, num_of_iter)

Trening modelu 2:

Iteracja: 100, strata: 0.167192
Iteracja: 200, strata: 0.147327
Iteracja: 300, strata: 0.122754
Iteracja: 400, strata: 0.127668
Iteracja: 500, strata: 0.123718
Iteracja: 600, strata: 0.119336
Iteracja: 700, strata: 0.115480
Iteracja: 800, strata: 0.112278


In [5]:
# Strategia Hi-Low

import json

f = open('strategy.json')
algo = json.load(f)
f.close()

In [9]:
# Symulacja rozgrywki

# Uczestnicy i ich strategie/modele
Pan_dealer = dealer()
Gracz1 = player(Pan_dealer, model = model_1)
Gracz2 = player(Pan_dealer, model = model_2)
Gracz3 = player(Pan_dealer, algorytm = algo)


# Statystyki
Wygrane1 = 0
Wygrane2 = 0
Wygrane3 = 0

Remisy1 = 0
Remisy2 = 0
Remisy3 = 0

LiczbaGier = 1000

# Wartosc prawdziwa metody Hi-Low
trueValue = 0

def setTrueValue(val):
    global trueValue
    if val < 7:
        trueValue += 1
    elif val > 9:
        trueValue -= 1
        
# Funkcja pilnujaca porzadek przy grze
def Graj(gracz):
    decyzja = 1
    while(not gracz.isBusted() and gracz.hand < 21 and decyzja == 1):
        decyzja = gracz.move(trueValue)
        if decyzja == 1:
            setTrueValue(gracz.hit())

# Przebieg rozgrywki
for i in range(LiczbaGier):
    # Tasowanie kart
    Pan_dealer.shuffleDeck()
    trueValue = 0
    Gracz1.reset()
    Gracz2.reset()
    Gracz3.reset()
    
    # Rozdanie kart
    setTrueValue(Pan_dealer.drawOne())
    
    setTrueValue(Gracz1.hit())
    setTrueValue(Gracz2.hit())
    setTrueValue(Gracz3.hit())
    setTrueValue(Gracz1.hit())
    setTrueValue(Gracz2.hit())
    setTrueValue(Gracz3.hit())
    
    # Gra
    if i%3 == 1:
        Graj(Gracz1)
        Graj(Gracz2)
        Graj(Gracz3)
    elif 1%3 == 2:
        Graj(Gracz2)
        Graj(Gracz3)
        Graj(Gracz1)
    else:
        Graj(Gracz3)
        Graj(Gracz1)
        Graj(Gracz2)
    
    Pan_dealer.drawTillFin()
    
    # Podliczanie wyników
    
    print('Rozdanie:', i+1)
    print('Gracz1: ', Gracz1.hand)
    print('Gracz2: ', Gracz2.hand)
    print('Gracz3: ', Gracz3.hand)
    print('Dealer: ', Pan_dealer.hand)
    print()
    
    if(not Gracz1.isBusted() and (Pan_dealer.hand > 21 or Gracz1.hand > Pan_dealer.hand)):
        Wygrane1 += 1
    elif(not Gracz1.isBusted() and (Gracz1.hand == Pan_dealer.hand)):
        Remisy1 += 1
    elif(not Gracz2.isBusted() and (Pan_dealer.hand > 21 or Gracz2.hand > Pan_dealer.hand)):
        Wygrane2 += 1
    elif(not Gracz2.isBusted() and (Gracz2.hand == Pan_dealer.hand)):
        Remisy2 += 1
    elif(not Gracz3.isBusted() and (Pan_dealer.hand > 21 or Gracz3.hand > Pan_dealer.hand)):
        Wygrane3 += 1
    elif(not Gracz3.isBusted() and (Gracz3.hand == Pan_dealer.hand)):
        Remisy3 += 1

print()
print('Gracz 1:')
print('Wygrane:', Wygrane1, 'na', LiczbaGier)
print('Remisy:', Remisy1, 'na', LiczbaGier)
print('Procent nieprzegranych:', (Wygrane1 + Remisy1)/LiczbaGier*100, '%')
print()
print()

print('Gracz 2:')
print('Wygrane:', Wygrane2, 'na', LiczbaGier)
print('Remisy:', Remisy2, 'na', LiczbaGier)
print('Procent nieprzegranych:', (Wygrane2 + Remisy2)/LiczbaGier*100, '%')
print()
print()

print('Gracz 3:')
print('Wygrane:', Wygrane3, 'na', LiczbaGier)
print('Remisy:', Remisy3, 'na', LiczbaGier)
print('Procent nieprzegranych:', (Wygrane3 + Remisy3)/LiczbaGier*100, '%')

Rozdanie: 1
Gracz1:  19
Gracz2:  15
Gracz3:  20
Dealer:  19

Rozdanie: 2
Gracz1:  19
Gracz2:  13
Gracz3:  17
Dealer:  20

Rozdanie: 3
Gracz1:  25
Gracz2:  15
Gracz3:  21
Dealer:  17

Rozdanie: 4
Gracz1:  20
Gracz2:  16
Gracz3:  20
Dealer:  20

Rozdanie: 5
Gracz1:  24
Gracz2:  17
Gracz3:  17
Dealer:  23

Rozdanie: 6
Gracz1:  19
Gracz2:  14
Gracz3:  22
Dealer:  22

Rozdanie: 7
Gracz1:  22
Gracz2:  23
Gracz3:  20
Dealer:  20

Rozdanie: 8
Gracz1:  21
Gracz2:  19
Gracz3:  23
Dealer:  19

Rozdanie: 9
Gracz1:  21
Gracz2:  15
Gracz3:  21
Dealer:  21

Rozdanie: 10
Gracz1:  19
Gracz2:  21
Gracz3:  21
Dealer:  17

Rozdanie: 11
Gracz1:  27
Gracz2:  15
Gracz3:  21
Dealer:  17

Rozdanie: 12
Gracz1:  15
Gracz2:  16
Gracz3:  15
Dealer:  22

Rozdanie: 13
Gracz1:  21
Gracz2:  15
Gracz3:  18
Dealer:  19

Rozdanie: 14
Gracz1:  24
Gracz2:  20
Gracz3:  20
Dealer:  17

Rozdanie: 15
Gracz1:  15
Gracz2:  19
Gracz3:  15
Dealer:  24

Rozdanie: 16
Gracz1:  24
Gracz2:  14
Gracz3:  21
Dealer:  18

Rozdanie: 17
Grac

Rozdanie: 656
Gracz1:  20
Gracz2:  18
Gracz3:  20
Dealer:  25

Rozdanie: 657
Gracz1:  22
Gracz2:  15
Gracz3:  19
Dealer:  26

Rozdanie: 658
Gracz1:  15
Gracz2:  16
Gracz3:  20
Dealer:  18

Rozdanie: 659
Gracz1:  26
Gracz2:  15
Gracz3:  22
Dealer:  17

Rozdanie: 660
Gracz1:  20
Gracz2:  15
Gracz3:  15
Dealer:  21

Rozdanie: 661
Gracz1:  20
Gracz2:  15
Gracz3:  21
Dealer:  19

Rozdanie: 662
Gracz1:  9
Gracz2:  19
Gracz3:  20
Dealer:  24

Rozdanie: 663
Gracz1:  19
Gracz2:  18
Gracz3:  23
Dealer:  18

Rozdanie: 664
Gracz1:  21
Gracz2:  13
Gracz3:  18
Dealer:  18

Rozdanie: 665
Gracz1:  17
Gracz2:  17
Gracz3:  19
Dealer:  18

Rozdanie: 666
Gracz1:  23
Gracz2:  21
Gracz3:  18
Dealer:  18

Rozdanie: 667
Gracz1:  17
Gracz2:  15
Gracz3:  26
Dealer:  19

Rozdanie: 668
Gracz1:  21
Gracz2:  18
Gracz3:  20
Dealer:  17

Rozdanie: 669
Gracz1:  23
Gracz2:  24
Gracz3:  16
Dealer:  19

Rozdanie: 670
Gracz1:  21
Gracz2:  15
Gracz3:  13
Dealer:  22

Rozdanie: 671
Gracz1:  14
Gracz2:  13
Gracz3:  14
Dealer